In [1]:
from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.apg import train as apg
from brax.training.agents.apg import networks as apg_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap
import jax.random
from jax import lax

In [2]:
from utils.SimpleConverter import SimpleConverter
from agent_mimic_env.agent_template import HumanoidTemplate
from agent_mimic_env.agent_eval_template import HumanoidEvalTemplate
from utils.util_data import *
from copy import deepcopy

In [3]:
import agent_mimic_env
from agent_mimic_env import register_mimic_env


In [4]:

import distutils.util
import os
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl


In [5]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [6]:
import yaml
from box import Box
# Path to your YAML file
yaml_file_path = 'config_params/punch.yaml'
# Load the YAML file
with open(yaml_file_path, 'r') as file:
    args = Box(yaml.safe_load(file))

In [7]:
print(args)

{'num_envs': 256, 'num_eval_envs': 32, 'lr': 0.0003, 'max_it': 1000, 'max_grad_norm': 0.4, 'seed': 0, 'system_config': 'humanoid', 'demo_replay_mode': 'threshold', 'threshold': 0.5, 'normalize_observations': True, 'cycle_len': 0, 'ep_len': 30, 'ep_len_eval': 54, 'use_lr_scheduler': True, 'reward_scaling': 0.02, 'rot_weight': 0.5, 'vel_weight': 0.01, 'ang_weight': 0.01, 'reward_weights': {'w_p': 0.6, 'w_v': 0.1, 'w_e': 0.2, 'w_c': 0.1}, 'ref': 'motions/humanoid3d_punch.txt'}


In [8]:
args.reward_weights.w_p

0.6

In [9]:
from agent_mimic_env.pds_controllers_agents import feedback_pd_controller, stable_pd_controller_action


In [10]:
env_replay,env_eval,env=register_mimic_env(args)


#for the env
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)
jit_step_custom_env = jax.jit(env.step_custom)
env.set_pd_callback(stable_pd_controller_action)


#for the eval
jit_reset_eval = jax.jit(env_eval.reset)
jit_step_custom_replay = jax.jit(env_eval.step_custom)
env_eval.set_pd_callback(stable_pd_controller_action)

#for the replay
jit_reset_replay = jax.jit(env_replay.reset)
jit_step_replay= jax.jit(env_replay.step)





In [11]:
print(env_eval.rollout_lenght)
print(env_eval.cycle_len)
print(env_eval.rot_weight)
print(env_eval.vel_weight)
print(env_eval.ang_weight)
print(env_eval.reward_scaling)
print(env_eval.dt)
print(env_eval.sys.dt)

print(env.rollout_lenght)
print(env.cycle_len)
print(env.rot_weight)
print(env.vel_weight)
print(env.ang_weight)
print(env.reward_scaling)
print(env.dt)
print(env.sys.dt)
print(env.err_threshold)

65
65
0.5
0.01
0.01
0.02
0.016
0.002
65
65
0.5
0.01
0.01
0.02
0.016
0.002
0.5


In [17]:
# initialize the state
#for the eval
# state = jit_reset_eval(jax.random.PRNGKey(0))
# rollout = [state.pipeline_state]
#for env
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]
# grab a 500 steps
n_steps = 500

In [18]:
for i in range(200):
    
    #print(i)
    #ctrl = -0.1 * jp.ones(env_eval.sys.nu)
    ctrl = -0.1 * jp.ones(env.sys.nu)
    #time
    #time = state.pipeline_state.time
    
    print('time: ',state.pipeline_state.time)    
         
    #state = jit_step_custom_replay(state, ctrl)
    #state = jit_step(state, ctrl)
    state = jit_step_custom_env(state,ctrl)
    #print(state.pipeline_state.geom_xpos)
    #if state.done:
     #   print(time)
      #  break
    # print("Is done", state.done)
    # print("Is rewards", state.reward)
    # print("Is ", state.metrics['pose_error'])
    
    rollout.append(state.pipeline_state)

time:  0.0
Replay: 0.0
time:  0.016
Replay: 0.0
time:  0.032
Replay: 0.0
time:  0.048000004
Replay: 0.0
time:  0.064
Replay: 0.0
time:  0.079999976
Replay: 0.0
time:  0.09599995
Replay: 0.0
time:  0.11199992
Replay: 0.0
time:  0.1279999
Replay: 0.0
time:  0.14399993
Replay: 0.0
time:  0.15999997
Replay: 0.0
time:  0.176
Replay: 0.0
time:  0.19200003
Replay: 0.0
time:  0.20800006
Replay: 0.0
time:  0.2240001
Replay: 0.0
time:  0.24000013
Replay: 0.0
time:  0.25600016
Replay: 0.0
time:  0.2720002
Replay: 0.0
time:  0.28800023
Replay: 0.0
time:  0.30400026
Replay: 1.0
time:  0.3200003
Replay: 0.0
time:  0.33600032
Replay: 0.0
time:  0.35200036
Replay: 0.0
time:  0.3680004
Replay: 0.0
time:  0.38400042
Replay: 0.0
time:  0.40000045
Replay: 0.0
time:  0.4160005
Replay: 0.0
time:  0.43200052
Replay: 0.0
time:  0.44800055
Replay: 1.0
time:  0.46400058
Replay: 0.0
time:  0.48000062
Replay: 0.0
time:  0.49600065
Replay: 0.0
time:  0.5120005
Replay: 0.0
time:  0.5280003
Replay: 0.0
time:  0.5440

In [14]:
from some_math.quaternion_diff import *

In [15]:
#6 right writs 9 left writs 12 right ankle, 15 left ankle

print(state.pipeline_state.geom_xpos)


#index 16

[[0.         0.         0.        ]
 [1.6105936  1.3574803  0.6877399 ]
 [1.6923536  1.4781059  0.92869556]
 [1.7937053  1.6215037  1.142208  ]
 [1.9506106  1.4519712  0.88578004]
 [2.0305326  1.473556   0.7220083 ]
 [2.066032   1.5859443  0.6484219 ]
 [1.4460938  1.4452083  1.0496316 ]
 [1.4031221  1.4036927  0.9185075 ]
 [1.4646019  1.4577059  0.8062173 ]
 [1.5776509  1.4012473  0.4172721 ]
 [1.409283   1.2888455  0.19321936]
 [1.308791   1.1116942  0.10089993]
 [1.3772587  1.3438598  0.48955715]
 [1.2147838  1.1629055  0.25728902]
 [1.1546856  0.9544053  0.14747915]]


In [19]:
HTML(html.render(env_replay.sys.replace(dt=env_replay.dt), rollout))